In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import tukey
import ipywidgets as widgets

# Set inline
%matplotlib inline

In [2]:
def CW_pulse_propagation(p, t, tv, frequency, tau, win):
    """
    Function to compute continous wave pulse using Tukey window.
    """
    # Compute the full pulse with the phase factor:
    CW_pulse = p * np.exp(-1j * (2*np.pi*frequency*t - np.pi/2))
    
    # Identify indices where the pulse is active: (t - tv) in [0, tau]
    ind = (t - tv >= 0) & (t - tv <= tau)
    
    # Zero out values outside the active pulse window
    CW_pulse[(t - tv) < 0] = 0
    CW_pulse[(t - tv) > tau] = 0
    
    if np.any(ind):
        # Apply the Tukey window
        window = tukey(np.sum(ind), win)
        CW_pulse[ind] *= window
    
    return CW_pulse


In [3]:
def lloyd_acoustic_field(plot_type, frequency, source_depth):
    """
    Lloyd's mirror acoustic field visualization.
    """
    # Sound speed
    c = 1500

    # Source level
    SL = 0

    # Convert SL (dB) to pressure amplitude (µPa)
    A_SL = 10**(SL/20)

    # Horizontal extent (m)
    Lx = 50

    # Vertical extent (m)
    Ly = 20

    # Number of horizontal points
    Nx = 150

    # Number of vertical points
    Ny = Nx

    # Create domain
    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)
    X, Y = np.meshgrid(x, y)
    
    # Compute ranges to the real and virtual sources
    r1 = np.sqrt(X**2 + (Y - source_depth)**2)
    r2 = np.sqrt(X**2 + (Y + source_depth)**2)
    
    # Calculate pressures from the two sources
    eps = 1e-6
    p1 = A_SL * np.exp(1j * 2*np.pi*frequency*r1/c) / r1
    p2 = -A_SL * np.exp(1j * 2*np.pi*frequency*r2/c) / r2
    p = p1 + p2 + eps

    # Plot pressure, intensity, or phase
    fig, ax = plt.subplots(figsize=(8, 6))
    if plot_type == "Pressure":
        data = np.real(p) * np.sqrt(2)
        label = 'Pressure (µPa)'
    elif plot_type == "Intensity":
        data = 20 * np.log10(np.abs(p))
        label = 'Intensity (dB re 1 µPa)'
    elif plot_type == "Phase":
        data = np.angle(p)
        label = 'Phase (radians)'
    c = plt.pcolormesh(x, y, data, shading='auto', cmap='viridis')
    if plot_type == "Intensity":
        c.set_clim(-50, 10)
    cbar = fig.colorbar(c, ax=ax)
    cbar.set_label(label)
    ax.set_title("Acoustic Field")
    ax.set_xlabel('x (M)')
    ax.set_ylabel('y (M)')
    ax.set_ylim(1, 20)

    # Show plot
    plt.tight_layout()
    plt.show()

# Create the interactive widgets
layout = widgets.Layout(width='500px')
widgets.interact(lloyd_acoustic_field,
    plot_type=widgets.Dropdown(
        options=["Pressure", "Intensity", "Phase"],
        value="Intensity",
        description="Plot Type",
        style={'description_width': 'initial'}, 
        layout=layout,
    ),
    frequency=widgets.IntSlider(
        min=100,
        max=1000,
        step=5,
        value=500,
        description='Frequency (Hz)',
        style={'description_width': 'initial'}, 
        layout=layout,
    ),
    source_depth=widgets.FloatSlider(
        min=1,
        max=19.9,
        step=0.1,
        value=10,
        description='Source Depth',
        style={'description_width': 'initial'}, 
        layout=layout,
    ),
)

interactive(children=(Dropdown(description='Plot Type', index=1, layout=Layout(width='500px'), options=('Press…

<function __main__.lloyd_acoustic_field(plot_type, frequency, source_depth)>

In [4]:
def lloyd_pulse_propagation(pulse_width):
    """
    Lloyd's mirror pulse propagation visualization.
    """
    # Frequency (Hz)
    frequency =  500

    # Source depth
    source_depth = 10

    # Sound speed (m/s)
    c = 1500

    # Source level (dB)
    SL = 0

    # Receiver depth (m)
    recDepth = 15

    # Receiver range (m)
    recRange = 3

    # Convert SL (dB) to pressure amplitude (µPa)
    A_SL = 10**(SL/20)
    
    # Calculate distances from receiver to real and virtual sources
    rec1R = np.sqrt(recRange**2 + (recDepth - source_depth)**2)
    rec2R = np.sqrt(recRange**2 + (recDepth + source_depth)**2)
    ts1 = rec1R / c
    ts2 = rec2R / c
    
    # Calculate pressures for direct and reflected paths
    pr1 = A_SL * np.exp(1j * 2*np.pi*frequency*rec1R/c) / rec1R
    pr2 = -A_SL * np.exp(1j * 2*np.pi*frequency*rec2R/c) / rec2R
    intensityCW = 20 * np.log10(np.abs(pr1 + pr2))
    
    # Create the time axis for simulation
    dt = (1/frequency)/20
    t1 = np.arange(0, ts2*3 + dt, dt)

    # Duration of transmitted pulse
    tau = pulse_width / frequency
    
    # Compute the transmitted pulses for both paths
    pulse1 = CW_pulse_propagation(pr1, t1, ts1, frequency, tau, 0.2)
    pulse2 = CW_pulse_propagation(pr2, t1, ts2, frequency, tau, 0.2)
    Output_pulse = pulse1 + pulse2 + 1e-6
    
    # Create subplots
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

    # Create pressure plot
    ax1.plot(t1, np.real(Output_pulse) * np.sqrt(2) * 10**(SL/20))
    ax1.set_xlim(0, 0.05)
    ax1.set_ylim(-0.35, 0.35)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Pressure (µPa)')
    ax1.set_title("Pressure Time Series")

    # Create intensity plot
    ax2.plot(t1, 20 * np.log10(np.abs(Output_pulse)) + SL, linewidth=2)
    ax2.plot([t1[0], t1[-1]], [intensityCW, intensityCW], 'k--', label='CW Intensity')
    ax2.set_xlim(0, 0.05)
    ax2.set_ylim(-45, -10)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Intensity (dB)')
    ax2.set_title("Intensity Time Series")
    ax2.legend()

    # Show plot
    plt.tight_layout()
    plt.show()

# Create the interactive widget
layout = widgets.Layout(width='500px')
widgets.interact(lloyd_pulse_propagation, 
    pulse_width=widgets.FloatSlider(
        min=1,
        max=15,
        step=0.1,
        value=3.0,
        description='Pulse Width',
        style={'description_width': 'initial'}, 
        layout=layout,
    )
)

interactive(children=(FloatSlider(value=3.0, description='Pulse Width', layout=Layout(width='500px'), max=15.0…

<function __main__.lloyd_pulse_propagation(pulse_width)>